# Lecture 04 – part II
                                       
## Introduction to Data-munging
                                       
   - Add new manipulated variables     
   - Separate char to new variables    
   - Convert variables to numeric or factor               
   - Some string manipulations         
   - Rename variables                  
   - Filter out different observations 
     - conditional selection           
     - tabulate frequency of a var     
     - missing values                  
     - replace values                  
     - duplicates                      
   - Using pipes                
   - Sorting data                      
                                       
#### Case-study 

- Based on: Chapter 02, A:              
    - Finding a good deal among hotels: data preparation              
                                      
#### Dataset:                              
- hotels-europe 
---

In [ ]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

Import raw data

In [ ]:
raw_df = pd.read_csv("https://osf.io/yzntm/download")

Have a look at the data

In [ ]:
raw_df.head()

### 1) Adding a new variable

Let us add nnights variable, which shows the number of nights spent in the hotel\
As the data was collected in such way, it is 1 for each observations.

You can assign a new variable using the following syntax

In [ ]:
raw_df["nnights"] = 1

Or use the `assign` function of pandas and also create a new variable, lets call it `df`

In [ ]:
df = raw_df.assign(nnights = 1)

Lets remove the raw data

In [ ]:
del raw_df

### 2) Data cleaning - selecting a variable

You can select variables with `[]` brackets after the DataFrame object

In [ ]:
df[["accommodationtype" , "price"]]

Or use the `filter` function

In [ ]:
df.filter(["accommodationtype" , "price"])

You can also select columns with `filter` using Regular Expressions aka regex

In [ ]:
df.filter(regex="rating")

### 3) separating character vector with a unique sign 

 Check accomotationtype: it is a character with a clear separator "@"

 To clean accommodationtype column: separate the characters at @ 
   and create two new variables: "garbage" and "acc_type".
 garbage will contain all characters before @ sign and acc_type will take everything after!

In [ ]:
df["accommodationtype"]

You can easily split and strip character columns wiht pandas with attaching the `str` function after column name

In [ ]:
df["accommodationtype"].str.split("@")

This results in a Series of lists, each list contains the splitted parts of the original string. You can access the with the following syntax

In [ ]:
df["accommodationtype"].str.split("@").str[0]

In [ ]:
df["accommodationtype"].str.split("@").str[1].str.strip()

The second value in the lists contains the type of information about the accomodation type, lets create a new variable out of it.

In [ ]:
df["acc_type"] = df["accommodationtype"].str.split("@").str[1].str.strip()
df["acc_type"].value_counts()

Its a good idea to use the `strip` function when splitting string variables as it removes any unnecessary white space

### Task - creating a numeric vector w simple separation

  1) Correct the `guestreviewsrating` variable into simple numeric variable\
  2) Check with `.dtypes` method\
  3) Convert the variable into a numeric variable


In [ ]:
df["guestreviewsrating"].value_counts(dropna=False)

In [ ]:
df["ratings"] = df["guestreviewsrating"].str.split('/').str[0].str.strip()
df["ratings"].dtypes

In [ ]:
df["ratings"] = df["ratings"].astype(float)

In [ ]:
df["ratings"].dtypes

### Task

Create numerical variables from the two distance measures:

In [ ]:
df[["center1distance", "center2distance"]]

We have two numeric values than the format of the distance "miles"

In [ ]:
df["distance"] = df["center1distance"].str.strip().str.split().str[0].astype(float)
df["distance_alter"] = df["center2distance"].str.strip().str.split().str[0].astype(float)

Check results

In [ ]:
df["distance"].hist(bins=20)

### Rename variables

Using the `rename` function, you have to specify in a dictionary that which columns you want to rename and their new names

In [ ]:
df.filter(regex="rating")

In [ ]:
df = df.rename(columns={"rating_reviewcount": "rating_count", "rating2_ta": "ratingta"})

In [ ]:
df.filter(regex="rating")

### Task:
   
also rename the following variables as follows:
     
    rating2_ta_reviewcount: ratingta_count
    addresscountryname: "country
    starrating: stars
    s_city": city

In [ ]:
df = df.rename(columns={
    "rating2_ta_reviewcount": "ratingta_count",
    "addresscountryname": "country",
    "starrating": "stars",
    "s_city": "city",
})

### Filtering observations

Use the `loc` function

In [ ]:
df.shape

let us have only hotels

In [ ]:
df.loc[df["acc_type"] == "Hotel"]

Filtering: find missing values\
look at one of our key variable: ratings\
we can calculate the frequencies of the ratings


In [ ]:
df["ratings"].isnull().sum()

What can we do with the NA values?\
First check them with 'loc'


In [ ]:
df.loc[df["ratings"].isnull()]

if reasonable we can drop them, but there needs to be good reason for that!


In [ ]:
df = df.loc[df["ratings"].notnull()]

Or alternatively ose `dropna`

In [ ]:
df = df.dropna(subset=["ratings"])

In [ ]:
df

### Correcting wrongly documented observations:
 In case of `stars` there are only values from 0-5

In [ ]:
df["stars"].value_counts(dropna=False)

What does 0 star means? It is missing, but recorded as 0...\
We need to set these values to `None`: re-define the stars variable:

In [ ]:
df["stars"] = np.where(df["stars"] == 0, None, df["stars"])

In [ ]:
df["stars"].value_counts(dropna=False)

### Duplicates:

1.) exact match for each values for a given observations\
 
Count the number of duplicates

In [ ]:
df.duplicated().sum()

Remove them

In [ ]:
df = df.drop_duplicates()

2.) Remove duplicates to specific variables, that are important to us

To make sense, let us take this into two steps: 

    a) select certain variables:


In [ ]:
important_variables = [
    "country",
    "hotel_id",
    "distance",
    "stars",
    "ratings",
    "price",
    "year",
    "month",
    "weekend",
    "holiday",
]

 Now we can filter out only duplicates in these variables:

In [ ]:
df = df.drop_duplicates(subset=important_variables)

### Task: Get specific data used in DA1 course:
       1) Get hotels only from Vienna
       2) Filter out the following observations:
           - in date: 2017, November and 0s week (multiple conditions)
           - with Hotel types which has stars between 3 and 4
           - and drop observations which has price more than 1000 EUR.


In [ ]:
hotel_vienna = df.loc[df["city"] == "Vienna"]
hotel_vienna = hotel_vienna.loc[
    (hotel_vienna["year"] == 2017)
    & (hotel_vienna["month"] == 11)
    & (hotel_vienna["weekend"] == 0)
]  # use | and & for bitwise operations
hotel_vienna = hotel_vienna.loc[hotel_vienna["acc_type"] == "Hotel"]
hotel_vienna = hotel_vienna.loc[
    (hotel_vienna["stars"] >= 3) & (hotel_vienna["stars"] <= 4)
]
hotel_vienna = hotel_vienna.loc[hotel_vienna["price"] < 1000]

In [ ]:
hotel_vienna

### Pipes with pandas

You can also do the above procedure in a pipe, which allows to make multiple sequential manipulations with one command:

In [ ]:
(
    df.loc[df["city"] == "Vienna"]
    .loc[(df["year"] == 2017) & (df["month"] == 11) & (df["weekend"] == 0)]
    .loc[df["acc_type"] == "Hotel"]
    .loc[(df["stars"] >= 3) & (df["stars"] <= 4)]
    .loc[df["price"] < 1000]
)

with pipeing, you can lace several different functions in one pipe, eg.

In [ ]:
(
    df.loc[(df["year"] == 2017) & (df["month"] == 11) & (df["weekend"] == 0)]
    .rename(columns={"city": "town"})
    .filter(["hotel_id", "country", "town", "price", "acc_type"])
    .groupby("town")
    .agg(n_hotels=("hotel_id", "nunique"), mean_price=("price", "mean"))
    .sort_values(by=["mean_price"], ascending=False)
    .head(10)
)

Make data table more "pretty"

Can arrange the values in increasing order

In [ ]:
hotel_vienna.sort_values(by=["price"], ascending=True)

Or decreasing order

In [ ]:
hotel_vienna.sort_values(by=["price"], ascending=False)

### Task: writing out csv as clean data

In [ ]:
hotel_vienna.to_csv("hotel_vienna_restricted.csv", index=False)